# Convirtiendo datos a ttl
:D


In [242]:
import pandas as pd # no quitar, se rompe el proyectoimport pandas as pd

In [243]:
#Leemos TODO PORQUE SI
circuits = pd.read_csv('data/circuits.csv')
constructorResults = pd.read_csv('data/constructor_results.csv')
constructors = pd.read_csv('data/constructors.csv')
constructorStandings = pd.read_csv('data/constructor_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
driverStandings = pd.read_csv('data/driver_standings.csv')
lapTimes = pd.read_csv('data/lap_times.csv')
pitStops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')
NAME_FILE = "data.ttl"

In [244]:
print("nueva, notacion usando las siguientes data classes, esta para \n1 mantener la ontologia limpia es decir no tener duplicas de definicion de algo \n2 hacer un solo flush en el archivo ttl los datos, lo que implicara una mejora en el tiempo de la generacion del archivo")

from dataclasses import dataclass, field
from typing import List, Union, Dict

@dataclass
class RDFPredicate:
    property: str
    objects: List[str]

@dataclass
class RDFStatement:
    _instances = {}
    subject: str
    predicates: dict[str, list[str]] = field(default_factory=dict)

    def __new__(cls, *args, **kwargs):
        # subject expected to either be the first args or with the key "subject"
        subject = kwargs.get("subject", args[0] if args else None)

        if not subject:
            raise ValueError("RDFStatement requires a 'subject' argument.")
    
        if subject not in cls._instances:
            new_instance = super(RDFStatement, cls).__new__(cls)
            new_instance.subject = subject
            new_instance.predicates = {}
            cls._instances[subject] = new_instance
        
        return cls._instances[subject]

    def __init__(self, subject: str, predicate: Union[str, RDFPredicate, List[RDFPredicate], List[List[any]]] = None, objects: Union[str, List[str]] = None):
        if predicate is None and objects is None:
            return
        
        elif (all(isinstance(p, list) for p in predicate)):
            for p in predicate:
                self._insert_statement(p[0], p[1])
        
        else:
            self._insert_statement(predicate, objects)

    def _insert_statement(self, predicate: Union[str, RDFPredicate, List[RDFPredicate], List[List[RDFPredicate]]]= None, objects: Union[str, List[str]] = None):
        if isinstance(predicate, str):
            is_objects_str = isinstance(objects, str)
            if not (is_objects_str or all(isinstance(o, str) for o in objects)):
                raise Exception(f"RDFStatement: {type(objects)} expected Union[str, List[str]]")
            if is_objects_str:
                objects = [objects]
            self._insert_on_property(predicate, objects)
        
        elif isinstance(predicate, RDFPredicate):
            self._insert_on_property(predicate.property, predicate.objects)

        elif isinstance(predicate, list):
            if not all(isinstance(p, RDFPredicate) for p in predicate):
                raise Exception(f"RDFStatement: {type(predicate)=} expected List[RDFPredicate]")
            for rdfp in predicate:
                self._insert_on_property(rdfp.property, rdfp.objects)
        elif predicate is None and objects is None:
            return
        else:
            raise Exception(f"RDFStatement:{predicate} {type(predicate)=} expected Union[str, RDFPredicate, List[RDFPredicate]]")
        
    def _insert_on_property(self, property: str, objects: list[str]):
        error = ["a", "rdf:type", "rdfs:type"]
        if property in error:
            if any(e in self.predicates.keys() for e in error):
                print(f"RDFStatement: Redefinition of a object type. {self} intended to overwrite {property} with {objects}")
                # raise Exception(f"RDFStatement: Redefinition of a object type.")
        if property in self.predicates:
            for obj in objects:
                if obj not in self.predicates[property]:
                    self.predicates[property].append(obj)
        else:
            self.predicates[property] = objects
    
    def to_ttl(self) -> str:
        ret = self.subject + " "
        for property, objects in self.predicates.items():
            ret += property + " " + ", ".join(objects) + ";\n\t"
        ret = ret[:-3]
        ret += "."
        return ret
    
    def __hash__(self):
        return id(self)
    
    def __eq__(self, other):
        if isinstance(other, RDFStatement):
            return self.__hash__() == other.__hash__()
        return False
    
class OrderSet:
    def __init__(self):
        self.data = {} 

    def insert(self, element):
        if element not in self.data:
            self.data[element] = len(self.data)

    def get_yield(self):
        # Devolver los elementos con su orden de inserción
        for element, order in self.data.items():
            yield (order, element)

ontology = OrderSet()
subjects = OrderSet()

nueva, notacion usando las siguientes data classes, esta para 
1 mantener la ontologia limpia es decir no tener duplicas de definicion de algo 
2 hacer un solo flush en el archivo ttl los datos, lo que implicara una mejora en el tiempo de la generacion del archivo


Ej 1:
```ttl
:11 a rdfs:Class;
	:famosoPor "numero chistoso"@es, "es primo"@es.
```

In [245]:
RDFStatement(":11", "a", "rdfs:Class")
RDFStatement(":11", ":famosoPor", ['"numero chistoso"@es', '"es primo"@es'])
print(RDFStatement(":11").to_ttl())

:11 a rdfs:Class;
	:famosoPor "numero chistoso"@es, "es primo"@es.


Ej 2:
```ttl
f1:Circuit a rdf:Class, owl:Class;
    rdfs:label 'A F1 Circuit'@en;
    rdfs:comment 'A class representing a F1 circuit'@en;
```

In [246]:
print(RDFStatement("f1:Circuit", [["a", ["rdf:Class", "owl:Class"]],
                            ["rdfs:label", "'A F1 Circuit'@en"],
                            ["rdfs:comment", "'A class representing a F1 circuit'@en"]]).to_ttl())

f1:Circuit a rdf:Class, owl:Class;
	rdfs:label 'A F1 Circuit'@en;
	rdfs:comment 'A class representing a F1 circuit'@en.


In [247]:
import unicodedata
def ttl_format(input_str: str) -> str:
    input_str = ''.join(c for c in unicodedata.normalize('NFKD', input_str) if not unicodedata.combining(c))
    input_str = input_str.lower().title().replace(" ", "_")
    return input_str

# Generar RDF Nationality

In [248]:
# TODO: HACER LA WEA
ontology.insert(RDFStatement("f1:Nationality", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 nationality'@en"],
                                            ["rdfs:comment", "'A class representing a nationality related to F1'@en"]]))

unique_nationality = drivers['nationality'].unique()

for nationality in unique_nationality:
    subjects.insert(RDFStatement(f"f1:{ttl_format(nationality)}", [
        ["a", "f1:Nationality"],
        ["rdfs:label", f"'{nationality}'@en"]
    ]))

# Generar RDF Drivers

In [249]:
# Tenemos las nacionalidades de los pilotos, ahora crearemos el resto de datos

# Creamos la clase f1 Driver

ontology.insert(RDFStatement("f1:Driver", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:label", "'A F1 Driver'@en"],
    ["rdfs:comment", "'A class representing a F1 driver'@en"]
]))

ontology.insert(RDFStatement("f1:Code", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:label", "'A F1 Short 3 letter display'@en"],
    ["rdfs:comment", "'The usual way to represent a competitor in a race'@en"]
]))

# Añadimos cada driver con su nacionalidad correspondiente:
# Por cada driver, añadimos su nombre, su fecha de nacimiento, su nacionalidad y su número de piloto

#Muchos drivers number son simplemente /N, eliminar?
driver = []
for index, row in drivers.iterrows():
    driver.append(RDFStatement(f"f1:{ttl_format(row['driverRef'])}", [
        ["a", "f1:Driver"],
        ["rdfs:label", f'"{row['forename']} {row['surname']}"@en'],
        ["f1:Nationality",f"f1:{ttl_format(row['nationality'])}"],
        ["f1:dateOfBirth", f"'{row['dob']}'^^xsd:date"]
    ]))
    if row['code'] != r'\N':
        driver.append(RDFStatement(f"f1:{ttl_format(row['driverRef'])}", "f1:Code", f"'{row['code']}'"))
        

for d in driver:
    subjects.insert(d)


RDFStatement: Redefinition of a object type. RDFStatement(subject='f1:Driver', predicates={'a': ['rdf:Class', 'owl:Class'], 'rdfs:label': ["'A F1 Driver'@en"], 'rdfs:comment': ["'A class representing a F1 driver'@en"]}) intended to overwrite a with ['f1:Driver']


# Generar RDF Pistas

## Ontology needed

```ttl
f1:Circuit a rdf:Class, owl:Class;
    rdfs:label 'A F1 Circuit'@en;
    rdfs:comment 'A class representing a F1 circuit'@en.

f1:City a rdf:Class, owl:Class;
    rdfs:label 'A City'@en;
    rdfs:comment 'A City in the world related to f1'@en.

f1:hostCity a rdf:Property;
    rdfs:label 'City'@en;
    rdfs:comment 'a City hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit.

f1:hostCountry a rdf:Property;
    rdfs:label 'Country'@en;
    rdfs:comment 'a Country hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit.
```

In [250]:
ontology.insert(RDFStatement("f1:Circuit", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 Circuit'@en"],
                                            ["rdfs:comment", "'A class representing a F1 circuit'@en"]]))

ontology.insert(RDFStatement("f1:Country", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 Country'@en"],
                                            ["rdfs:comment", "'A class representing a F1 country'@en"]]))

ontology.insert(RDFStatement("f1:City",[["a", ["rdf:Class", "owl:Class"]],
                                        ["rdfs:label", "'A City'@en"],
                                        ["rdfs:comment", "'A City in the world related to f1'@en;"]]))

ontology.insert(RDFStatement("f1:hostCity",[["a", "rdf:Property"],
                                            ["rdfs:label", "'City'@en"],
                                            ["rdfs:comment", "'a City hosting a F1 circuit'@en"],
                                            ["rdfs:domain", "f1:Circuit"]]))

ontology.insert(RDFStatement("f1:hostCountry", [["a", "rdf:Property"],
                                                ["rdfs:label", "'Country'@en"],
                                                ["rdfs:comment", "'a Country hosting a F1 circuit'@en"],
                                                ["rdfs:domain", "f1:Circuit"]]))

RDFStatement: Redefinition of a object type. RDFStatement(subject='f1:Circuit', predicates={'a': ['rdf:Class', 'owl:Class'], 'rdfs:label': ["'A F1 Circuit'@en"], 'rdfs:comment': ["'A class representing a F1 circuit'@en"]}) intended to overwrite a with ['rdf:Class', 'owl:Class']


* Example:
```ttl
f1:albert_park a f1:Circuit;
    rdfs:label 'Albert Park Grand Prix Circuit'@en;
    f1:hostCity f1:Melbourne;
    f1:hostCountry f1:Australia;
    geo:lat "-37.8497"^^xsd:float;
    geo:long "144.968"^^xsd:float;
```

In [251]:
cities = []
countries = []
sub1 = []
sub2 = []
for index, row in circuits.iterrows():
    sub2.append(
        RDFStatement(
            f"f1:{ttl_format(row["circuitRef"])}_Circuit",
            [
                ["rdf:type", "f1:Circuit"],
                ["rdfs:label", f"'{row["name"]}'@en"],
                ["f1:hostCity", f"f1:{ttl_format(row["location"])}"],
                ["f1:hostCountry", f"f1:{ttl_format(row["country"])}"],
                ["geo:lat", f'"{row["lat"]}"^^xsd:float'],
                ["geo:long", f'"{row["lng"]}"^^xsd:float']
            ]
        )
    )
    cities.append(ttl_format(row["location"]))
    countries.append(ttl_format(row["country"]))

cities = list(set(cities))
for city in cities:
    sub1.append(RDFStatement(f"f1:{city}", "a", "f1:City"))

countries = list(set(countries))
for country in countries:
    sub1.append(RDFStatement(f"f1:{country}", "a", "f1:Country"))

for i in sub1:
    subjects.insert(i)

for i in sub2:
    subjects.insert(i)


# Generar RDF Constructores

In [ ]:
ontology.insert(RDFStatement("f1:Constructor", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'A class representing an F1 constructor'@en"]
    ]))

constructors_RDFS = []
for index,row in constructors.iterrows():
    constructors_RDFS.append(RDFStatement(f"f1:{ttl_format(row['constructorRef'])}_Constructor", [
        ["a", "f1:Constructor"],
        ["rdfs:label", f"'{row['name']}'@en"],
        ["f1:Nationality", f"f1:{row["nationality"].replace(" ", "_")}"]
        ]))

for c in constructors_RDFS:
    subjects.insert(c)
        

# Escritura de ontologia y subjects

In [253]:
with open(NAME_FILE, 'w') as f:
    f.write("""@prefix f1: <http://ex.org/f1#>.
@prefix geo:  <http://www.opengis.net/ont/geosparql#>
@prefix owl: <http://www.w3.org/2002/07/owl#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>. 
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.\n\n""")

    l = list(ontology.get_yield())
    l.sort()
    for e in l:
        f.write(e[1].to_ttl() + "\n\n")

    l = list(subjects.get_yield())
    l.sort()
    for e in l:
        f.write(e[1].to_ttl() + "\n\n")